# oauth2

> For oauth2

In [3]:
#| default_exp oauth2

In [4]:
!which python

/home/uts/aishiftscheduler/.venv/bin/python


In [5]:
!python --version

Python 3.10.12


In [6]:
# Set up autoreload
%load_ext autoreload
%autoreload 2

In [7]:
#| hide
from nbdev.showdoc import *

In [10]:
#| export
from jose import JWTError, jwt
from datetime import datetime, timedelta
import aishiftscheduler.schemas as sch
import aishiftscheduler.database as db
import aishiftscheduler.dbmodels as dbm
from fastapi import Depends, status, HTTPException
from fastapi.security import OAuth2PasswordBearer
from sqlalchemy.orm import Session

oauth2_scheme = OAuth2PasswordBearer(tokenUrl="login")

# import aishiftscheduler.dbmodels as dbm
# import aishiftscheduler.schemas as sch
# import aishiftscheduler.utils as utl
# from aishiftscheduler.database import get_db

/home/uts/aishiftscheduler/aishiftscheduler/shift_scheduler_data.xlsx
/home/uts/aishiftscheduler/aishiftscheduler/best_theta_data.xlsx
/home/uts/aishiftscheduler/aishiftscheduler/AIShiftScheduler1.png

len(thNAMES)=5
thNAMES=['thCumSlots', 'thSickProb', 'thCumMerits', 'thContSlots', 'thSelect']


In [ ]:
#| export
## SECRET_KEY
## Algorithm
## Expiration time

SECRET_KEY = "change me by going to oauth2 docs and get longer one"
ALGORITHM = "HS256"
ACCESS_TOKEN_EXPIRE_MINUTES = 60

def create_access_token(data: dict):
    to_encode = data.copy()

    # expire = datetime.utcnow() + timedelta(minutes=ACCESS_TOKEN_EXPIRE_MINUTES)
    expire = datetime.utcnow() + timedelta(minutes=ACCESS_TOKEN_EXPIRE_MINUTES)
    to_encode.update({"exp": expire})

    encoded_jwt = jwt.encode(to_encode, SECRET_KEY, algorithm=ALGORITHM)

    return encoded_jwt

In [ ]:
#| export
def verify_access_token(token: str, credentials_exception):
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        id: str = payload.get("user_id")
        if id is None:
            raise credentials_exception
        token_data = sch.TokenData(id=id)
    # except JWTError as e:
    except JWTError:
        # print(e)
        raise credentials_exception
    # except AssertionError as e:
    return token_data

In [ ]:
#| export
def get_current_user(token: str=Depends(oauth2_scheme), db: Session=Depends(db.get_db)):
    credentials_exception = HTTPException(status_code=status.HTTP_401_UNAUTHORIZED, detail=f"Could not validate credentials", headers={"WWW-Authenticate": "Bearer"})
    # return verify_access_token(token, credentials_exception)
    token = verify_access_token(token, credentials_exception)
    user = db.query(dbm.User).filter(dbm.User.id == token.id).first()
    return user